# Convert Laz to DEM using CloudCompare

# Required software

+ CloudCompare
+ QGIS
+ GoogleColab

## Process flow

https://jeffblackadar.github.io/maryland_rch/maryland_rch-lidar.html

## File work areas

There are two file works areas
+ (local drive) local PC: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files
```
+ (Google Drive) Google Drive: 
```
/content/drive/MyDrive/crane_maryland/
```

### Download files from Google Drive to local drive

There is a constraint that only a 2gb zip file can be downloaded at one time. The .laz files are large so they have been batched into units of 50.
Download each batch.

### Local processing

The local PC has CloudCompare and QGIS installed, these are not available on Google Colab so it's necessary to moved files from Google Drive and process them on a PC and then move them back. Manual tasks but I could not find a better means to process these files using this collection of software.

On the local PC laz files will be converted to DEM using CloudCompare. CloudCompare is not installed on Google Colab so it's used on the local PC.

4 batches of 50 laz files (ex: 0-49, 50-99, 100-149, 150-199) will be downloaed into a folder that can contain 200 files (ex: laz_utm_0-199).

Steps:

#### 1. Create folders to hold downladed files. Example: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_0-199. 
```

This can be done by running the first cell of 1_1_Maryland_download_laz_tiles_use_CloudCompare_to_convert_to_DEM (below)


In [ ]:
# Cell 1. Create working directories
# Run this only once
# set up working directories to hold downloaded files
import os
input_laz_utm_fp_base = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"

chunk_amount = 200
total_files = 2154

for count in range(0, total_files , 200):

    new_dir_file_path = os.path.join(input_laz_utm_fp_base, ("laz_utm_"+str(count)+"-"+str(count+chunk_amount-1)))
    print(new_dir_file_path)
    if not (os.path.exists(new_dir_file_path)):  
        print("Does not exist.")
        os.mkdir(new_dir_file_path)
    else:
        print("Exists.")
        
        
input_slope_fp_base = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\"
for count in range(0, total_files , 200):

    new_dir_file_path = os.path.join(input_slope_fp_base, ("slope_"+str(count)+"-"+str(count+chunk_amount-1)))
    print(new_dir_file_path)
    if not (os.path.exists(new_dir_file_path)):  
        print("Does not exist.")
        os.mkdir(new_dir_file_path)
    else:
        print("Exists.")


#### 2. Manually download the files from Google Drive. Enter the folder and select the files. Verify 50 items selected.
(example):

```
/content/drive/MyDrive/crane_maryland/laz/laz_fema_2012_forested_utm/batch_50/laz_utm_0-49 
```

![local_processing_step_2.png](local_processing_step_2.png)

#### 3. Unzip into directory the appropriate directory to hold 200 files
Extract all

(example) from the zip file of 50 files:

```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-249-20211114T195617Z-001.zip\laz_utm_200-249
```

(example) to the folder to hold a group of 200 files: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399
```

Download and unzip 4 groups of 50 files into each folder to contain 200 files. Check that the folder has 200 files at the end of each group.

## Run CloudCompare

In [59]:
# Cell 2 Run CloudCompare
# E:\Program Files\CloudCompare>cloudcompare -O 
#'E:/a_new_orgs/carleton/hist5706-maryland/lidar_files/reprojected/20120323_001415_r_utm.laz' 
#-SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1

import subprocess
import time

# example command
# C:\LAStools\bin\las2las -i "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\20120323_001460.laz" -target_utm 18north -o "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120323_001460_r_utm.laz"

batch_group = "1000-1599"
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"

counter = 0
import os
#process .laz files one at a time
for root, dirs, files in os.walk(input_laz_utm_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".laz"):
          laz_in_path = os.path.join(input_laz_utm_fp, name)
          #raster_out_path = os.path.join(output_raster_fp, (name[:-4]+".tif"))
          command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
          print(str(counter))        
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          #time.sleep(4)
          counter = counter + 1        
        

"""
Rasterizes the loaded clouds by projecting them inside a regular grid.
Optional settings are:

-VERT_DIR {0/1/2} to specify the projection dimension (0=X / 1=Y / 2=Z) - default is 2 (Z)
-PROJ {MIN/AVG/MAX} to specify how the altitude of each cell is computed (minimum, average or maximum altitude of the projected points). Default is AVG (average)
-SF_PROJ {MIN/AVG/MAX} to specify how the SF value(s) of each cell are computed (minimum, average or maximum altitude of the projected points SF value(s)). Default is AVG (average)
-EMPTY_FILL {MIN_H/MAX_H/CUSTOM_H/INTERP} to specify how empty cells should be filled (MIN_H = min height / MAX_H = max height / CUSTOM_H = custom value - see CUSTOM_HEIGHT option below / INTERP = interpolate). Default behavior is 'leave cells empty'
-CUSTOM_HEIGHT {value} to define the custom height filling value (if the 'CUSTOM_H' strategy is used (see above)
-OUTPUT_CLOUD to output the result as a cloud (this is the default option if no other output format is defined)
-OUTPUT_MESH to output the result as a mesh
-OUTPUT_RASTER_Z to output the result as a geotiff raster (only altitudes, no RGB)
-OUTPUT_RASTER_Z_AND_SF to output the result as a geotiff raster (altitudes + all SFs by default, no RGB)
-OUTPUT_RASTER_RGB to output the result as a geotiff raster (RGB)        
"""

0
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
1
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001949_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
2
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001984_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
3
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001985_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZ

32
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_002120_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
33
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_002131_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
34
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_002132_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
35
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_002141_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RAST

64
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001207_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
65
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001221_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
66
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001222_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
67
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001223_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RAST

96
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001340_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
97
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001341_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
98
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001368_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
99
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001369_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RAST

127
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001532_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
128
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001533_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
129
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001534_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
130
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001535_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -

...

499
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000531_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
500
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_001170_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
501
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_001171_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
502
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_001185_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -

530
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001236_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
531
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001262_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
532
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
533
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001264_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -

561
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001663_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
562
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001664_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
563
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001712_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
564
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001713_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -

592
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120404_001760_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
593
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120410_001810_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
594
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120412_001856_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
595
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120412_001899_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -

"\nRasterizes the loaded clouds by projecting them inside a regular grid.\nOptional settings are:\n\n-VERT_DIR {0/1/2} to specify the projection dimension (0=X / 1=Y / 2=Z) - default is 2 (Z)\n-PROJ {MIN/AVG/MAX} to specify how the altitude of each cell is computed (minimum, average or maximum altitude of the projected points). Default is AVG (average)\n-SF_PROJ {MIN/AVG/MAX} to specify how the SF value(s) of each cell are computed (minimum, average or maximum altitude of the projected points SF value(s)). Default is AVG (average)\n-EMPTY_FILL {MIN_H/MAX_H/CUSTOM_H/INTERP} to specify how empty cells should be filled (MIN_H = min height / MAX_H = max height / CUSTOM_H = custom value - see CUSTOM_HEIGHT option below / INTERP = interpolate). Default behavior is 'leave cells empty'\n-CUSTOM_HEIGHT {value} to define the custom height filling value (if the 'CUSTOM_H' strategy is used (see above)\n-OUTPUT_CLOUD to output the result as a cloud (this is the default option if no other output for

# Re-run a skipped command individually

In [ ]:
!"E:\\Program Files\\CloudCompare\\cloudcompare" -SILENT -O E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_200-399\\20120218_000003_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 

# Copy/Move tifs

CloudCompare creates the DEM tifs in the same directory as the las files. This program copies them to their own directory. Later, once this process is more repeatable, this will become a move, not a copy, to save disk space.

## !! Be careful and set the directories correctly.  Mixing batches together is annoying

In [62]:
# Cell 3 Copy/Move tifs
import os
import shutil
# Divide files into groups of 200 so they can be downloaded and processed in managably sized chunks.
file_counter = 0

batch_group = "1000-1599"
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

if not (os.path.exists(dem_tif_fp_dest)):  
    print("Does not exist: ", dem_tif_fp_dest)
    os.mkdir(dem_tif_fp_dest)

for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            shutil.move(origin_file_path, dem_tif_fp_dest)
            print(file_counter, origin_file_path)
            file_counter = file_counter + 1        

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm_2021-12-31_14h37_21_752.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm_2021-12-31_16h17_00_936.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h17_04_934.tif
0 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001948_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h17_04_934.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001949_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120310_001949_utm_2021-12-31_14h37_29_421.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001279_utm_2021-12-31_14h50_00_656.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001279_utm_2021-12-31_16h34_47_414.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001279_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h34_51_547.tif
79 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001279_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h34_51_547.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001280_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001280_utm_2021-12-31_14h50_09_520.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001280_utm_2021-12-31_16h35_00_467.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_f

151 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001731_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h51_36_348.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001732_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001732_utm_2021-12-31_15h01_31_985.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001732_utm_2021-12-31_16h51_45_769.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001732_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h51_49_460.tif
152 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001732_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_16h51_49_460.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120311_001774_utm.laz
E:\a_new_orgs\carleton\hist5706-marylan

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001255_utm_2021-12-31_15h15_10_602.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001255_utm_2021-12-31_17h10_06_948.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001255_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h10_10_861.tif
238 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001255_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h10_10_861.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001256_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001256_utm_2021-12-31_15h15_22_969.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001256_utm_2021-12-31_17h10_23_303.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001791_utm_2021-12-31_15h32_34_101.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001791_utm_2021-12-31_17h32_47_399.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001791_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h32_51_293.tif
324 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001791_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h32_51_293.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001792_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001792_utm_2021-12-31_15h32_41_114.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120312_001792_utm_2021-12-31_17h32_58_576.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_

...

426 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001417_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h54_32_192.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001418_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001418_utm_2021-12-31_15h51_00_463.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001418_utm_2021-12-31_17h54_43_604.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001418_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h54_47_623.tif
427 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001418_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_17h54_47_623.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001419_utm.laz
E:\a_new_orgs\carleton\hist5706-marylan

451 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001541_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h00_31_393.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001542_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001542_utm_2021-12-31_15h55_25_411.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001542_utm_2021-12-31_18h00_41_077.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001542_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h00_44_920.tif
452 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001542_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h00_44_920.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120323_001543_utm.laz
E:\a_new_orgs\carleton\hist5706-marylan

474 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000431_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h05_41_825.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000466_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000466_utm_2021-12-31_15h59_10_947.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000466_utm_2021-12-31_18h05_50_187.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000466_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h05_53_686.tif
475 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000466_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h05_53_686.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120330_000520_utm.laz
E:\a_new_orgs\carleton\hist5706-marylan

510 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001262_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h13_06_882.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm_2021-12-31_16h04_20_817.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm_2021-12-31_18h13_16_134.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h13_19_690.tif
511 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001263_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h13_19_690.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120401_001264_utm.laz
E:\a_new_orgs\carleton\hist5706-marylan

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001428_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h22_13_749.tif
554 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001428_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h22_13_749.tif
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001471_utm.laz
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001471_utm_2021-12-31_16h10_55_467.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001471_utm_2021-12-31_18h22_24_668.bin
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001471_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h22_28_475.tif
555 E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\20120402_001471_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-12-31_18h2

# Quality Assurance 1: Confirm we have 200 lazs and 200 DEM tifs with matching file names.

In [63]:
# Cell 4 Quality Assurance 1
# Dictionary of lazs
batch_group = "1000-1599"
expected_number_of_files = 600
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

lazs_dict = {}
for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".laz"):
            lazs_dict[file_name[:file_name.find("_utm")+4]]=file_name

# Dictionary of tifs
tifs_dict = {}
for file_name in os.listdir(dem_tif_fp_dest):
    origin_file_path = os.path.join(dem_tif_fp_dest,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            tifs_dict[file_name[:file_name.find("_utm")+4]]=file_name

counter = 0
print("There are:",len(lazs_dict),"laz files in", input_laz_utm_fp)
print("There are:",len(tifs_dict),"tif files in", dem_tif_fp_dest)
for dict_key in lazs_dict:
    if not dict_key in tifs_dict:
        print(lazs_dict[dict_key], " (laz) is not in tifs directory.")
        print("Let's try to process with a scalar field index of 6. For some Laz files, Classfication is at the index of 6, not 8.")
        laz_in_path = os.path.join(input_laz_utm_fp, lazs_dict[dict_key])
        # The index of the scalar field is 6 this time -SET_ACTIVE_SF 6
        command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
        print("file",str(counter), "processed.")
        print("!"+command)
        subprocess.call(command, shell=True)
        counter = counter + 1        


for dict_key in tifs_dict:
    if not dict_key in lazs_dict:
        print(tifs_dict[dict_key], "(tif) is not in lazs directory.")
      
if len(lazs_dict)==expected_number_of_files and len(tifs_dict)==expected_number_of_files:
    print("Both directories have",expected_number_of_files,"files. This looks good.")
else:
    print("There is a problem. Check the number of files in each folder.")

        

There are: 600 laz files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\
There are: 600 tif files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\dem_tifs\
Both directories have 600 files. This looks good.


# Open QGIS, Run:
```
1_2_qgis_load_dem_save_as_slope.py
```
This saves a slope file for each DEM tif in the diectory

```
1_3_qgis_save_slope_tif_as_jpg.py
```
This saves a jpg file for each slope tif in the diectory

# Quality Assurance 2: Confirm we have 200 lazs, 200 DEM tifs, 200 Slope tifs and 200 Slope jpgs with matching file names.

In [64]:
# Cell 5 Quality Assurance
# Dictionary of lazs

input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

lazs_dict = {}
for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".laz"):
            lazs_dict[file_name[:file_name.find("_utm")+4]]=file_name

# Dictionary of tifs
tifs_dict = {}
for file_name in os.listdir(dem_tif_fp_dest):
    origin_file_path = os.path.join(dem_tif_fp_dest,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            tifs_dict[file_name[:file_name.find("_utm")+4]]=file_name

counter = 0
print("There are:",len(lazs_dict),"laz files in", input_laz_utm_fp)
for dict_key in lazs_dict:
    if not dict_key in tifs_dict:
        print(lazs_dict[dict_key], "(laz) is not in tifs directory.")
        print("Go back to cell 4 and re-run the process. All of the lazs need to be converted to DEM tifs before running QGIS to create slope files.")
        print("file",str(counter), "processed.")
        counter = counter + 1        

print("There are:",len(tifs_dict),"tif files in", dem_tif_fp_dest)
for dict_key in tifs_dict:
    if not dict_key in lazs_dict:
        print(tifs_dict[dict_key], "(tif) is not in lazs directory.")        

# Dictionary of slope tifs
slope_tifs_dict = {}
slope_tifs_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\slope_"+batch_group+"\\"

for file_name in os.listdir(slope_tifs_fp):
    origin_file_path = os.path.join(slope_tifs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):
        if(origin_file_path[-4:]==".tif"):
            slope_tifs_dict[file_name[6:-4]]=file_name

print("There are:",len(slope_tifs_dict),"slope files in", slope_tifs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in tifs_dict:
        print(dict_key, "(slope tif) is not in DEM tifs directory.")
        
# Dictionary of slope jpgs
slope_jpgs_dict = {}
slope_jpgs_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\slope_"+batch_group+"\\jpgs\\"

for file_name in os.listdir(slope_jpgs_fp ):
    origin_file_path = os.path.join(slope_jpgs_fp ,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            slope_jpgs_dict[file_name[6:-4]]=file_name

print("There are:",len(slope_jpgs_dict),"slope jpg files in", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in slope_tifs_dict:
        print(dict_key, "(slope jpg) is not in slope tifs directory.")

if len(lazs_dict)==expected_number_of_files and len(tifs_dict)==expected_number_of_files and len(slope_tifs_dict)==expected_number_of_files and len(slope_jpgs_dict)==expected_number_of_files:
    print("All directories have",expected_number_of_files," files. This looks good.")
else:
    print("There is a problem. Check the number of files in each folder.")

There are: 600 laz files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\
There are: 600 tif files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1599\dem_tifs\
There are: 600 slope files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\slope_1000-1599\
There are: 600 slope jpg files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\slope_1000-1599\jpgs\
All directories have 600  files. This looks good.


# Upload jpg files to Google Drive

# XXXXXXXX Set crs of raster - Not yet working - Ignore below

In [19]:
import subprocess
import time
import rasterio

# example command
# C:\LAStools\bin\las2las -i "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\20120323_001460.laz" -target_utm 18north -o "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120323_001460_r_utm.laz"

input_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"
output_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\rasters\\"

# Set crs
import os
#process .tif files one at a time
for root, dirs, files in os.walk(input_raster_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".tif"):
          tif_out_path = os.path.join(input_raster_fp, name)
          tif_in_path = os.path.join(output_raster_fp, (name[:19]+"26918"+name_ext))
          with rasterio.open(tif_in_path, "w", crs='EPSG:26918') as dest:
              dest.write(out_image)         
          #command = 'gdal_edit.bat -a_srs EPSG:26985 "' + tif_in_path + '"'
                    
          #print("!"+command)
          #subprocess.call(command, shell=True)
          # just in case, let the sub process run
          time.sleep(4)            

ModuleNotFoundError: No module named 'rasterio'

In [20]:
!gdal_edit.bat -a_srs EPSG:26985 "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif"

'gdal_edit.bat' is not recognized as an internal or external command,
operable program or batch file.


# Make slope file - Not yet working

In [16]:
import subprocess
import time

# example command
# gdaldem slope E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif 
# C:/Users/User/AppData/Local/Temp/processing_QIEzpg/6c1743fc49544eba9ae8c25d891f86e8/OUTPUT.tif
# -of GTiff -b 1 -s 1.0


input_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"
output_slope_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\"


# Set crs
import os
#process .tif files one at a time
for root, dirs, files in os.walk(input_raster_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".tif"):
          tif_in_path = os.path.join(input_raster_fp, name)
          slope_out_path = os.path.join(output_slope_fp, (name[:-4]+"_slope"+name_ext))
        
          #command = '"C:\\Program Files\\QGIS 3.16\\bin\\gdaldem.exe" slope "' + tif_in_path + '" "' + slope_out_path + " -of GTiff -b 1 -s 1.0"
          command = '"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "' + tif_in_path + '" "' + slope_out_path + '" -of GTiff -b 1 -s 1.0'
            
            
                    
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          time.sleep(4)  

!"E:\Program Files\QGIS 3.10\bin\gdaldem" slope "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0


In [11]:
!"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0


0...10...20...30...40...50...60...70...80...90...100 - done.


In [15]:
!"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0

0...10...20...30...40...50...60...70...80...90...100 - done.


In [1]:
import os
import geopandas as gpd

lidar_tiles_forested_polys_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\hist5706-maryland\\fema_forested\\2012_fema_md_forested.shp"

"""
# Create an empty geopandas GeoDataFrame for duplicates
lidar_tiles_forested_polys_df = gpd.GeoDataFrame()
lidar_tiles_forested_polys_df['Index'] = None
lidar_tiles_forested_polys_df['Name'] = None
lidar_tiles_forested_polys_df['URL'] = None
lidar_tiles_forested_polys_df['geometry'] = None
"""

lidar_tiles_forested_polys_df.crs = ('EPSG:26985')

if os.path.exists(lidar_tiles_forested_polys_fp):
    lidar_tiles_forested_polys = gpd.read_file(lidar_tiles_forested_polys_fp)
    
    print("crs", lidar_tiles_forested_polys.crs)
    print(type(lidar_tiles_forested_polys))

    print("lidar_tiles_forested_polys",lidar_tiles_forested_polys.shape)
    # crs for Maryland
    area_crs = 26985
    land_use_polys.to_crs(area_crs)

    for index, row in lidar_tiles_forested_polys.iterrows():
       #print("row",row)

       print(row[1],row[2],row[3])

ModuleNotFoundError: No module named 'geopandas'

In [5]:
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_200-399\\"
#output_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\rasters\\"

counter = 0
import os
#process .laz files one at a time
for root, dirs, files in os.walk(input_laz_utm_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".laz"):
          laz_in_path = os.path.join(input_laz_utm_fp, name)
          #raster_out_path = os.path.join(output_raster_fp, (name[:-4]+".tif"))
          command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
          print("file",str(counter), "processed.")      
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          #time.sleep(4)
          counter = counter + 1     

0
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399\20120129_17SPD9981_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
1
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399\20120129_17SPD9983_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
2
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399\20120129_17SPE9701_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 
3
!"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399\20120129_17SQD0081_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RAST